In [54]:
import numpy as np
import tensorflow as tf 
import wave, math, struct
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation

In [55]:
# Prepare a dummy data for simulating motes 
notes_freqs = {
    'A':440.0,'B':493.88, 'C':261.63, 'D':293.66,
    'E':393.63, 'F':349.23, 'G': 392.00
}

In [56]:
notes_freqs

{'A': 440.0,
 'B': 493.88,
 'C': 261.63,
 'D': 293.66,
 'E': 393.63,
 'F': 349.23,
 'G': 392.0}

In [57]:
notes = list(notes_freqs.keys())

In [58]:
notes

['A', 'B', 'C', 'D', 'E', 'F', 'G']

In [59]:
note_to_int = {note: i for i, note in enumerate(notes)}

In [60]:
note_to_int

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}

In [61]:
int_to_note = {i: note for i, note in enumerate(notes)}

In [62]:
int_to_note

{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G'}

In [63]:
raw_music_data = [notes[np.random.randint(0,7)] for i in range(1000)]

In [64]:
raw_music_data

['E',
 'A',
 'E',
 'C',
 'B',
 'B',
 'E',
 'F',
 'E',
 'E',
 'G',
 'A',
 'E',
 'C',
 'D',
 'G',
 'C',
 'G',
 'G',
 'B',
 'F',
 'E',
 'E',
 'A',
 'B',
 'C',
 'F',
 'D',
 'A',
 'D',
 'F',
 'G',
 'E',
 'C',
 'D',
 'C',
 'D',
 'C',
 'G',
 'D',
 'F',
 'E',
 'F',
 'D',
 'D',
 'F',
 'G',
 'B',
 'A',
 'G',
 'F',
 'D',
 'D',
 'A',
 'E',
 'C',
 'D',
 'A',
 'G',
 'G',
 'C',
 'F',
 'E',
 'F',
 'E',
 'C',
 'C',
 'F',
 'B',
 'F',
 'A',
 'F',
 'G',
 'F',
 'B',
 'B',
 'C',
 'F',
 'A',
 'G',
 'A',
 'C',
 'A',
 'B',
 'B',
 'D',
 'B',
 'D',
 'G',
 'C',
 'F',
 'A',
 'F',
 'E',
 'B',
 'B',
 'D',
 'A',
 'D',
 'G',
 'C',
 'F',
 'B',
 'E',
 'E',
 'C',
 'B',
 'G',
 'A',
 'E',
 'B',
 'F',
 'D',
 'D',
 'B',
 'D',
 'D',
 'E',
 'E',
 'A',
 'B',
 'F',
 'B',
 'F',
 'D',
 'D',
 'C',
 'C',
 'F',
 'A',
 'E',
 'F',
 'A',
 'G',
 'F',
 'G',
 'E',
 'G',
 'E',
 'F',
 'A',
 'E',
 'G',
 'B',
 'E',
 'G',
 'B',
 'B',
 'F',
 'D',
 'B',
 'D',
 'A',
 'G',
 'E',
 'D',
 'B',
 'E',
 'A',
 'D',
 'F',
 'A',
 'E',
 'F',
 'E',
 'E',
 'C'

#### Data Preparation

In [65]:
sequence_length = 3
network_input = []
network_output = []

for i in range(len(raw_music_data) - sequence_length):
    seq_in = raw_music_data[i: i+sequence_length]
    seq_out = raw_music_data[i+sequence_length]
    network_input.append([note_to_int[char] for char in seq_in])
    network_output.append(note_to_int[seq_out])
    print(seq_in,'-->', seq_out)

['E', 'A', 'E'] --> C
['A', 'E', 'C'] --> B
['E', 'C', 'B'] --> B
['C', 'B', 'B'] --> E
['B', 'B', 'E'] --> F
['B', 'E', 'F'] --> E
['E', 'F', 'E'] --> E
['F', 'E', 'E'] --> G
['E', 'E', 'G'] --> A
['E', 'G', 'A'] --> E
['G', 'A', 'E'] --> C
['A', 'E', 'C'] --> D
['E', 'C', 'D'] --> G
['C', 'D', 'G'] --> C
['D', 'G', 'C'] --> G
['G', 'C', 'G'] --> G
['C', 'G', 'G'] --> B
['G', 'G', 'B'] --> F
['G', 'B', 'F'] --> E
['B', 'F', 'E'] --> E
['F', 'E', 'E'] --> A
['E', 'E', 'A'] --> B
['E', 'A', 'B'] --> C
['A', 'B', 'C'] --> F
['B', 'C', 'F'] --> D
['C', 'F', 'D'] --> A
['F', 'D', 'A'] --> D
['D', 'A', 'D'] --> F
['A', 'D', 'F'] --> G
['D', 'F', 'G'] --> E
['F', 'G', 'E'] --> C
['G', 'E', 'C'] --> D
['E', 'C', 'D'] --> C
['C', 'D', 'C'] --> D
['D', 'C', 'D'] --> C
['C', 'D', 'C'] --> G
['D', 'C', 'G'] --> D
['C', 'G', 'D'] --> F
['G', 'D', 'F'] --> E
['D', 'F', 'E'] --> F
['F', 'E', 'F'] --> D
['E', 'F', 'D'] --> D
['F', 'D', 'D'] --> F
['D', 'D', 'F'] --> G
['D', 'F', 'G'] --> B
['F', 'G',

In [66]:
n_patterns = len(network_input)

In [67]:
n_patterns

997

In [68]:
x = np.reshape(network_input,(n_patterns, sequence_length, 1))
x

array([[[4],
        [0],
        [4]],

       [[0],
        [4],
        [2]],

       [[4],
        [2],
        [1]],

       ...,

       [[0],
        [4],
        [2]],

       [[4],
        [2],
        [0]],

       [[2],
        [0],
        [4]]])

In [69]:
from keras.utils import to_categorical

In [70]:
y = to_categorical(network_output)

In [71]:
y.shape

(997, 7)

In [72]:
y

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

#### Build the model

In [73]:
model = Sequential()
model.add(LSTM(256, input_shape=(x.shape[1], x.shape[2])))
model.add(Dense(1000, activation='relu'))
model.add(Dense(7, activation='softmax'))

/home/mitu/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [74]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

#### Train the model

In [75]:
model.fit(x, y, epochs=100)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 1.9649
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.9522
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.9387
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.9344
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.9355
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.9393
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.9375
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.9333
Epoch 9/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.9367
Epoch 10/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.9300
Epoch 11/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.9301
Epoch 12/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.9230
Epoch 13/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.9311
Epoch 14/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.9347
Epoch 15/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss:

#### Generate a new melody sequence

In [76]:
start_index = np.random.randint(0, len(network_input))

In [77]:
pattern = network_input[start_index]

In [78]:
pattern

[2, 6, 5]

In [79]:
generated_melody = []
for i in range(16):
    x_input = np.reshape(pattern,(1,len(pattern),1))
    prediction = model.predict(x_input, verbose=False)
    index = np.argmax(prediction)
    result = int_to_note[index]
    generated_melody.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [80]:
pattern

[0, 1, 3]

In [81]:
generated_melody

['F',
 'E',
 'F',
 'E',
 'B',
 'B',
 'D',
 'B',
 'C',
 'E',
 'A',
 'A',
 'E',
 'A',
 'B',
 'D']

#### Save this as audio file

In [85]:
with wave.open('my_music.wav','w') as wav_file:
    wav_file.setparams((1, 2, 44100, 0,'NONE','not compressed'))
    for note in generated_melody:
        freq = notes_freqs[note]
        num_samples = int(0.5 * 44100) # duration * sample rate
        for i in range(num_samples):
            # sample rate
            t = float(i) / 44100
            value = int(32767 * 0.5 * math.sin(2*math.pi*freq*t))
            data = struct.pack('<h', value)
            wav_file.writeframes(data)

In [86]:
help(wave.open)

Help on function open in module wave:

open(f, mode=None)

